# Demographics and Employment in the United States

## Table of Content

[Data Description](#Data-Description)

[Loading and Summarizing the Dataset](#Loading-and-Summarizing-the-Dataset)

[Evaluating Missing Values](#Evaluating-Missing-Values)

[Integrating Metropolitan Area Data](#Integrating-Metropolitan-Area-Data)

[Integrating Country of Birth Data](#Integrating-Country-of-Birth-Data)

## Data Description

In the wake of the Great Recession of 2009, there has been a good deal of focus on employment statistics, one of the most important metrics policymakers use to gauge the overall strength of the economy. In the United States, the government measures unemployment using the Current Population Survey (CPS), which collects demographic and employment information from a wide range of Americans each month.

The observations in the dataset represent people surveyed in the September 2013 CPS who actually completed a survey. While the full dataset has 385 variables, in this exercise we will use a more compact version of the dataset, `CPSData.csv`, which has the following variables:

* **PeopleInHousehold**: The number of people in the interviewee's household.

* **Region**: The census region where the interviewee lives.

* **State**: The state where the interviewee lives.

* **MetroAreaCode**: A code that identifies the metropolitan area in which the interviewee lives (missing if the interviewee does not live in a metropolitan area). The mapping from codes to names of metropolitan areas is provided in the file MetroAreaCodes.csv.

* **Age**: The age, in years, of the interviewee. 80 represents people aged 80-84, and 85 represents people aged 85 and higher.

* **Married**: The marriage status of the interviewee.

* **Sex**: The sex of the interviewee.

* **Education**: The maximum level of education obtained by the interviewee.

* **Race**: The race of the interviewee.

* **Hispanic**: Whether the interviewee is of Hispanic ethnicity.

* **CountryOfBirthCode**: A code identifying the country of birth of the interviewee. The mapping from codes to names of countries is provided in the file CountryCodes.csv.

* **Citizenship**: The United States citizenship status of the interviewee.

* **EmploymentStatus**: The status of employment of the interviewee.

* **Industry**: The industry of employment of the interviewee (only available if they are employed).

## Loading and Summarizing the Dataset

In [1]:
import pandas as pd
print ('Pandas version: ', pd.__version__)

import numpy as np

import matplotlib.pyplot as plt

% matplotlib inline

Pandas version:  0.20.3


In [2]:
CPS = pd.read_csv("../Data/CPSData.csv")

In [3]:
CPS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131302 entries, 0 to 131301
Data columns (total 14 columns):
PeopleInHousehold     131302 non-null int64
Region                131302 non-null object
State                 131302 non-null object
MetroAreaCode         97064 non-null float64
Age                   131302 non-null int64
Married               105964 non-null object
Sex                   131302 non-null object
Education             105964 non-null object
Race                  131302 non-null object
Hispanic              131302 non-null int64
CountryOfBirthCode    131302 non-null int64
Citizenship           131302 non-null object
EmploymentStatus      105513 non-null object
Industry              66242 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 14.0+ MB


How many interviewees are in the dataset?



In [4]:
CPS.shape

(131302, 14)

Among the interviewees with a value reported for the Industry variable, what is the most common industry of employment? Please enter the name exactly how you see it.

In [5]:
CPS["Industry"].value_counts()

Educational and health services                15017
Trade                                           8933
Professional and business services              7519
Manufacturing                                   6791
Leisure and hospitality                         6364
Construction                                    4387
Financial                                       4347
Transportation and utilities                    3260
Other services                                  3224
Public administration                           3186
Information                                     1328
Agriculture, forestry, fishing, and hunting     1307
Mining                                           550
Armed forces                                      29
Name: Industry, dtype: int64

Which state has the fewest interviewees? Which state has the largest number of interviewees?

In [6]:
CPS["State"].value_counts()

California              11570
Texas                    7077
New York                 5595
Florida                  5149
Pennsylvania             3930
Illinois                 3912
Ohio                     3678
Maryland                 3200
Minnesota                3139
Michigan                 3063
Virginia                 2953
Colorado                 2925
Connecticut              2836
Georgia                  2807
Wisconsin                2686
New Hampshire            2662
North Carolina           2619
New Jersey               2567
Iowa                     2528
Washington               2366
Maine                    2263
Delaware                 2214
Rhode Island             2209
Missouri                 2145
Hawaii                   2099
Indiana                  2004
South Dakota             2000
Massachusetts            1987
Nebraska                 1949
Oregon                   1943
Kansas                   1935
Vermont                  1890
Nevada                   1856
Utah      

What proportion of interviewees are citizens of the United States?

In [7]:
citizens_USA = CPS["Citizenship"].value_counts().to_frame()
print(citizens_USA)

(citizens_USA.loc['Citizen, Native'] + citizens_USA.loc['Citizen, Naturalized'])/len(CPS["Citizenship"])

                      Citizenship
Citizen, Native            116639
Non-Citizen                  7590
Citizen, Naturalized         7073


Citizenship    0.942194
dtype: float64

The CPS differentiates between race (with possible values American Indian, Asian, Black, Pacific Islander, White, or Multiracial) and ethnicity. A number of interviewees are of Hispanic ethnicity, as captured by the Hispanic variable. For which races are there at least 250 interviewees in the CPS dataset of Hispanic ethnicity?

In [8]:
pd.crosstab(CPS["Race"], CPS["Hispanic"])

Hispanic,0,1
Race,,
American Indian,1129,304
Asian,6407,113
Black,13292,621
Multiracial,2449,448
Pacific Islander,541,77
White,89190,16731


## Evaluating Missing Values

Which variables have at least one interviewee with a missing (NA) value? 

In [9]:
columns_with_NA = CPS.isnull().sum(axis = 0)
columns_with_NA[columns_with_NA > 0]

MetroAreaCode       34238
Married             25338
Education           25338
EmploymentStatus    25789
Industry            65060
dtype: int64

Often when evaluating a new dataset, we try to identify if there is a pattern in the missing values in the dataset. For example, the `Married` variable being missing is related to the `Age` value for the interviewee.

In [10]:
CPS["Married_NA"] = CPS["Married"].isnull()
pd.crosstab(CPS["Age"], CPS["Married_NA"] ).head(10)


Married_NA,False,True
Age,,
0,0,1283
1,0,1559
2,0,1574
3,0,1693
4,0,1695
5,0,1795
6,0,1721
7,0,1681
8,0,1729


As mentioned in the variable descriptions, MetroAreaCode is missing if an interviewee does not live in a metropolitan area. Using the same technique as in the previous question, answer the following questions about people who live in non-metropolitan areas.

How many states had all interviewees living in a non-metropolitan area (aka they have a missing MetroAreaCode value)? How many states had all interviewees living in a metropolitan area? For this question, treat the District of Columbia as a state (even though it is not technically a state).

In [11]:
pd.crosstab(CPS["State"], CPS["MetroAreaCode"].isnull())

MetroAreaCode,False,True
State,,
Alabama,1020,356
Alaska,0,1590
Arizona,1327,201
Arkansas,724,697
California,11333,237
Colorado,2545,380
Connecticut,2593,243
Delaware,1696,518
District of Columbia,1791,0


Alaska and Wyoming have no interviewees living in a metropolitan area, and the District of Columbia, New Jersey, and Rhode Island have all interviewees living in a metro area.

Which region of the United States has the largest proportion of interviewees living in a non-metropolitan area?

In [12]:
#  MetroAreaCode is missing if an interviewee does not live in a metropolitan area
# define interviewees living out metropolian area
CPS["NonMetropolian"] = CPS["MetroAreaCode"].isnull()

CPS_metro = pd.crosstab(CPS["Region"], CPS["NonMetropolian"], normalize = 'index')
print(CPS_metro)

NonMetropolian     False     True 
Region                            
Midwest         0.652131  0.347869
Northeast       0.783762  0.216238
South           0.762156  0.237844
West            0.756337  0.243663


## Integrating Metropolitan Area Data

Codes like MetroAreaCode and CountryOfBirthCode are a compact way to encode factor variables with text as their possible values, and they are therefore quite common in survey datasets. In fact, all but one of the variables in this dataset were actually stored by a numeric code in the original CPS datafile.

When analyzing a variable stored by a numeric code, we will often want to convert it into the values the codes represent. To do this, we will use a dictionary, which maps the the code to the actual value of the variable. We have provided dictionaries MetroAreaCodes.csv and CountryCodes.csv, which respectively map MetroAreaCode and CountryOfBirthCode into their true values. Read these two dictionaries into data frames MetroAreaMap and CountryMap.

How many observations (codes for metropolitan areas) are there in MetroAreaMap?

In [13]:
MetroAreaMap = pd.read_csv("../Data/MetroAreaCodes.csv")
MetroAreaMap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 2 columns):
Code         271 non-null int64
MetroArea    271 non-null object
dtypes: int64(1), object(1)
memory usage: 4.3+ KB


How many observations (codes for countries) are there in CountryMap?

In [14]:
CountryMap = pd.read_csv("../Data/CountryCodes.csv")
CountryMap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 2 columns):
Code       149 non-null int64
Country    149 non-null object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


To merge in the metropolitan areas, we want to connect the field MetroAreaCode from the CPS data frame with the field Code in MetroAreaMap.

In [15]:
CPS = pd.merge(CPS, MetroAreaMap, left_on = "MetroAreaCode", how = "left", right_on = "Code")
CPS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131302 entries, 0 to 131301
Data columns (total 18 columns):
PeopleInHousehold     131302 non-null int64
Region                131302 non-null object
State                 131302 non-null object
MetroAreaCode         97064 non-null object
Age                   131302 non-null int64
Married               105964 non-null object
Sex                   131302 non-null object
Education             105964 non-null object
Race                  131302 non-null object
Hispanic              131302 non-null int64
CountryOfBirthCode    131302 non-null int64
Citizenship           131302 non-null object
EmploymentStatus      105513 non-null object
Industry              66242 non-null object
Married_NA            131302 non-null bool
NonMetropolian        131302 non-null bool
Code                  97064 non-null float64
MetroArea             97064 non-null object
dtypes: bool(2), float64(1), int64(4), object(11)
memory usage: 17.3+ MB


How many interviewees have a missing value for the new metropolitan area variable? 

In [16]:
CPS["MetroArea"].isnull().sum()

34238

What metropolitan areas has the largest number of interviewees?

In [17]:
CPS["MetroArea"].value_counts().head(10)

New York-Northern New Jersey-Long Island, NY-NJ-PA    5409
Washington-Arlington-Alexandria, DC-VA-MD-WV          4177
Los Angeles-Long Beach-Santa Ana, CA                  4102
Philadelphia-Camden-Wilmington, PA-NJ-DE              2855
Chicago-Naperville-Joliet, IN-IN-WI                   2772
Providence-Fall River-Warwick, MA-RI                  2284
Boston-Cambridge-Quincy, MA-NH                        2229
Minneapolis-St Paul-Bloomington, MN-WI                1942
Dallas-Fort Worth-Arlington, TX                       1863
Houston-Baytown-Sugar Land, TX                        1649
Name: MetroArea, dtype: int64

Which metropolitan area has the highest proportion of interviewees of Hispanic ethnicity?

In [18]:
Top5Metropolitan_hispanic = pd.crosstab(CPS["MetroArea"], CPS["Hispanic"], normalize = "index").sort_values(by = 0).head()
Top5Metropolitan_hispanic

Hispanic,0,1
MetroArea,,
"Laredo, TX",0.033708,0.966292
"McAllen-Edinburg-Pharr, TX",0.051282,0.948718
"Brownsville-Harlingen, TX",0.202532,0.797468
"El Paso, TX",0.209016,0.790984
"El Centro, CA",0.313131,0.686869


 Determine the number of metropolitan areas in the United States from which at least 20% of interviewees are Asian.

In [19]:
frame = pd.crosstab(CPS["MetroArea"], CPS["Race"], normalize = "index")

frame[frame["Asian"] > 0.2]

Race,American Indian,Asian,Black,Multiracial,Pacific Islander,White
MetroArea,,,,,,
"Honolulu, HI",0.003807,0.501904,0.011421,0.194797,0.117386,0.170685
"San Francisco-Oakland-Fremont, CA",0.018759,0.246753,0.071429,0.045455,0.010823,0.606782
"San Jose-Sunnyvale-Santa Clara, CA",0.025373,0.241791,0.016418,0.040299,0.008955,0.667164
"Vallejo-Fairfield, CA",0.000000,0.203008,0.165414,0.015038,0.022556,0.593985


Determine which metropolitan area has the smallest proportion of interviewees who have received no high school diploma.

In [20]:
CPS["Education_NoDiploma"] = CPS["Education"] == "No high school diploma"
CPS["Education_NoDiploma"].groupby(CPS["MetroArea"]).mean().sort_values().head()

MetroArea
Iowa City, IA               0.022901
Bowling Green, KY           0.034483
Kalamazoo-Portage, MI       0.039370
Champaign-Urbana, IL        0.040984
Bremerton-Silverdale, WA    0.045977
Name: Education_NoDiploma, dtype: float64

## Integrating Country of Birth Data

Just as we did with the metropolitan area information, merge in the country of birth information from the CountryMap data frame, replacing the CPS data frame with the result.

What is the name of the variable added to the CPS data frame by this merge operation?

In [21]:
#CPS = pd.read_csv("../Data/CPSData.csv")
print(CPS.shape)

CPS = pd.merge(CPS, CountryMap, how = 'left', left_on = "CountryOfBirthCode", right_on = "Code")
print(CPS.shape)
CPS.info()

(131302, 19)
(131302, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 131302 entries, 0 to 131301
Data columns (total 21 columns):
PeopleInHousehold      131302 non-null int64
Region                 131302 non-null object
State                  131302 non-null object
MetroAreaCode          97064 non-null object
Age                    131302 non-null int64
Married                105964 non-null object
Sex                    131302 non-null object
Education              105964 non-null object
Race                   131302 non-null object
Hispanic               131302 non-null int64
CountryOfBirthCode     131302 non-null int64
Citizenship            131302 non-null object
EmploymentStatus       105513 non-null object
Industry               66242 non-null object
Married_NA             131302 non-null bool
NonMetropolian         131302 non-null bool
Code_x                 97064 non-null float64
MetroArea              97064 non-null object
Education_NoDiploma    131302 non-null bool
Co

How many interviewees have a missing value for the new country of birth variable?

In [22]:
CPS["Country"].isnull().sum()

176

Among all interviewees born outside of North America, which country was the most common place of birth?

In [28]:
CPS["Country"].value_counts().head()

United States    115063
Mexico             3921
Philippines         839
India               770
China               581
Name: Country, dtype: int64